# 반복 단어, 문장 1개로 통일

In [73]:
!pip install rouge

  Obtaining dependency information for rouge from https://files.pythonhosted.org/packages/32/7c/650ae86f92460e9e8ef969cc5008b24798dcf56a9a8947d04c78f550b3f5/rouge-1.0.1-py3-none-any.whl.metadata


In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from copy import deepcopy
import warnings
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
import rouge

In [2]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [3]:
train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')
filtered_train_df = make_dataframe('./train.json')
filtered_dev_df = make_dataframe('./dev.json')
filtered_test_df = make_dataframe('./test.json')

In [4]:
train_df

,id,conversation,subject_keyword,speakers,output
0,nikluge-2024-일상 대화 요약-train-000001,"[{'speaker': 'SD2000001', 'utterance': '저는 여행 ...",[해외여행],"[SD2000001, SD2000002]",이 대화에서 화자들은 좋았던 여행지와 기억나는 주요 명소에 대해 이야기했습니다. S...
1,nikluge-2024-일상 대화 요약-train-000002,"[{'speaker': 'SD2000002', 'utterance': '여행 다닐 ...","[여행 스타일, 숙소, 음식]","[SD2000002, SD2000001]","이 대화에서 화자들은 쇼핑하기, 숙소, 음식 등 각자 선호하는 여행 스타일에 대해 ..."
2,nikluge-2024-일상 대화 요약-train-000003,"[{'speaker': 'SD2000001', 'utterance': '어~ 그래서...","[여행지, 일화]","[SD2000001, SD2000002]",이 대화에서 화자들은 기억에 남는 여행지와 일화에 대해 이야기했습니다. SD2000...
3,nikluge-2024-일상 대화 요약-train-000004,"[{'speaker': 'SD2000005', 'utterance': 'name1 ...","[음식, 선호, 길거리 음식, 싫어하는 음식]","[SD2000005, SD2000006]",두 화자는 이 대화에서 좋아하는 음식과 싫어하는 음식에 대해 말했습니다. SD200...
4,nikluge-2024-일상 대화 요약-train-000005,"[{'speaker': 'SD2000006', 'utterance': '그러면 na...","[다이어트 식품, 마실것]","[SD2000005, SD2000006]",두 화자는 이 대화에서 다이어트 식품과 좋아하는 음료에 대해 이야기했습니다. SD2...
...,...,...,...,...,...
501,nikluge-2024-일상 대화 요약-train-000502,"[{'speaker': 'SD2100596', 'utterance': '그 comp...",[company-name5],"[SD2100596, SD2100595]",두 화자는 이 대화에서 유명 유튜버의 식당 추천과 음식 장사의 성공 요인에 대해서 ...
502,nikluge-2024-일상 대화 요약-train-000503,"[{'speaker': 'SD2100598', 'utterance': '나 이번 달...",[행복 주택],"[SD2100598, SD2100597]",이 대화에서 화자들은 주택 마련 방법과 고충에 대해서 이야기했습니다. SD21005...
503,nikluge-2024-일상 대화 요약-train-000504,"[{'speaker': 'SD2100598', 'utterance': '저는 지금 ...",[수익],"[SD2100598, SD2100597]",이 대화에서 화자들은 현재 소득에서 수익 창출 방법에 대해 이야기 했습니다. SD2...
504,nikluge-2024-일상 대화 요약-train-000505,"[{'speaker': 'SD2100599', 'utterance': '동네에 맛있...",[점심],"[SD2100599, SD2100600]",이 대화에서 화자들은 좋아하는 치킨 브랜드와 어제 저녁 먹은 음식과 점심으로 먹은 ...


In [5]:
cosmos = make_dataframe('../results/cosmos.json')

## output 내의 반복되는 단어

In [8]:
# output 내에 '\b([가-힣a-zA-Z0-9_]+)\s+\1\b'를 만족하는 부분이 있는지 확인하고 있다면 set에 추가하고 마지막에 return하는 함수

def find_repeated_words(df: pd.DataFrame, pattern) -> set:
    """
    Find repeated words in the 'output' column of the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame to search for repeated words.
    pattern (str): Pattern to search for.

    Returns:
    set: Set of repeated words.
    """
    count = 0

    repeated_words = set()
    for output in df['output']:
        matches = re.findall(pattern, output)
        repeated_words.update(matches)
        count+=len(matches)

    print(f"Total number of repeated words found: {len(repeated_words)}")
    print(f"Total number of repeated words found: {count}")

    return repeated_words


In [15]:
repeated_words = find_repeated_words(filtered_train_df, r'\b(\w+)\b(?:\s+\1\b)+')
repeated_words

Total number of repeated words found: 7
Total number of repeated words found: 8


{'가도', '그리고', '깜짝', '두루', '좋은', '할머니가', '화자'}

In [16]:
repeated_words = find_repeated_words(filtered_dev_df, r'\b(\w+)\s+\1\b')

Total number of repeated words found: 0
Total number of repeated words found: 0


## output 내의 반복되는 문장

In [6]:
# output 내의 반복되는 문장 찾기
rouge_score = rouge.Rouge().get_scores

def find_repeated_sentence(df: pd.DataFrame) -> set:
    """
    Find repeated sentences in the DataFrame.

    Parameters:
    df (pd.DataFrame): DataFrame of the output.

    Returns:
    set: Set of repeated sentences.
    """
    cnt = 0
    repeated_sentences = set()
    for output in df['output']:
        for sentence in output.split('.')[:-1]: # 마지막은 빈 문자열이므로 제외
            if output.count(sentence) > 1:
                repeated_sentences.add(sentence)
                cnt+=1

    print(f"Total number of repeated sentences found: {cnt}")
    print(f"the number of repeated sentences found: {len(repeated_sentences)}")

    return repeated_sentences

In [7]:
repeated_sentences = find_repeated_sentence(filtered_train_df)

Total number of repeated sentences found: 0
the number of repeated sentences found: 0


In [8]:
repeated_sentences = find_repeated_sentence(filtered_dev_df)

Total number of repeated sentences found: 0
the number of repeated sentences found: 0


In [55]:
repeated_sentences

{'SD2000862는 엔시티의 보스 착장 중 제복 착장이 멤버들의 피지컬 덕분에 좋았고 터프하게 랩을 하는 마크의 모습이 인상깊었다고 말했습니다'}

In [10]:
repeated_sentences = find_repeated_sentence(cosmos)
repeated_sentences

Total number of repeated sentences found: 289
the number of repeated sentences found: 43


{' SD2000058은 가족 중에서 가장 건강에 염려스러운 것은 치매라고 말했습니다',
 ' SD2000059는 가족 중에서 가장 건강에 염려스러운 것은 고혈압이라고 말했습니다',
 ' SD2000061은 반려동물 경험에 대해 이야기했습니다',
 ' SD2000174는 1927는 극장에서 보는 것이 좋고, SD2000175는 1917는 롱테이크로 극장에서 보는 것이 좋다고 말했습니다',
 ' SD2000174는 영화를 보는데 무슨 느낌인지 구체적으로 모르겠는 영화를 별로 좋아하지 않다고 말했습니다',
 ' SD2000259는 가족은 피로 이어져서도 있고 모든 면이 닮아 있어서 큰 의미가 있다고 말했습니다',
 ' SD2000260은 가족은 혈연을 넘어서 소중한 사람들로 구성되어 있다고 말했습니다',
 ' SD2000282는 음식에 대한 선호도에 대해 말했습니다',
 ' SD2000283는 음식에 대한 선호도에 대해 말했습니다',
 ' SD2000566은 해외 여행으로는 영국이 기억에 남는다고 말했습니다',
 ' SD2000567은 국내 여행으로 강원도에서 가장 기억에 남는다고 말했습니다',
 ' SD2002391은 유아교육과는 싫어서 추천하고 싶지 않다고 말했습니다',
 ' 가을은 체중이 줄어들고 패션이 좋아서 장점이라고 말했습니다',
 ' 가족과 함께 가는다면 단양을 추천하고, 친구들과 함께 가는다면 어디든 좋을 것 같다고 말했습니다',
 ' 가족이 방문하면 반찬을 준비해야 하는데, 가족이 두 명이기 때문에 걱정은 덜 하다고 말했습니다',
 ' 가족이 방문하면 반찬을 준비해야 하는데, 가족이 방문하면 반찬을 준비해야 하는데, 가족이 두 명이기 때문에 걱정은 덜 하다고 말했습니다',
 ' 가족이랑 가고 싶은 여행은 부모님이 돌아가시기 전에 꼭 가보고 싶다고 말했습니다',
 ' 간헐식하고 일주일에 10만보 걷기하고 그래서 지금 제가 몸에 대해서는 제 스스로 그냥 제 나이에 완벽하다고 생각을 하는데 유지를 꾸준히 하는 게 중요하고 이제는 안 먹어서 뺄 순 없잖아

In [71]:
# output의 맨 마지막이 '.'으로 끝나는지, '. '으로 끝나는지 확인
def check_last_character(df: pd.DataFrame) -> None:
    """
    Check if the last character of the 'output' column is '.' or '. '.

    Parameters:
    df (pd.DataFrame): DataFrame to check.
    """
    end_dot = 0
    end_dot_space = 0
    another = 0

    for output in df['output']:
        if output[-1] == '.':
            end_dot += 1
        elif output[-2:] == '. ':
            end_dot_space += 1
        else:
            another += 1
            print("another sentence : ",output)
        
    print(f"Number of outputs ending with '.': {end_dot}")
    print(f"Number of outputs ending with '. ': {end_dot_space}")
    print(f"Number of outputs ending with another character: {another}")

In [72]:
check_last_character(filtered_train_df)

another sentence :  두 화자는 이 대화에서 영화 관람 스타일에 대해 말했습니다. SD2001145는 영화관에 가서 신작을 봤으며 알아서 스스로 누르고 애플리케이션으로 들어가서 영화를 보면 되어서 영화관을 자주 이용한다고 말했습니다. 또 한국 영화 위주로 많이 보는데 요즘엔 외국 영화 중심으로 재개봉을 하고 있어 조금 아쉽다고 말했습니다. SD2001146은 코로나 때문에 영화관을 못 가서 신작 영화를 보진 못했고 넷플릭스나 웨이브에서 시청한다고 말했습니다. 집에서 티브이나 휴대폰으로 볼 땐 소리와 화면의 질이 아쉽다고 말했습니다
Number of outputs ending with '.': 498
Number of outputs ending with '. ': 7
Number of outputs ending with another character: 1


In [68]:
check_last_character(filtered_dev_df)

Number of outputs ending with '.': 100
Number of outputs ending with '. ': 2
Number of outputs ending with another character: 0
